# Вебинар 1. Введение, примеры задач, бизнес- и ML-метрики 

In [1]:
import pandas as pd
import numpy as np

# ML-mетрики качества

In [2]:
recommended_list = [1134, 156, 27, 991, 143, 1543, 3345, 533, 11, 43] #id товаров
prices_recommended = [30, 50, 40, 120, 70, 400, 60, 10, 240, 370]
bought_list = [521, 32, 143, 991]
prices_bought = [80, 150, 70, 120]

In [3]:
len(recommended_list)

10

### 1. Hit rate

Hit rate = был ли хотя бы 1 релевантный товар среди рекомендованных

- Иногда применяется, когда продаются достаточно дорогие товары (например, бытовая техника) 

----
Hit rate = (был ли хотя бы 1 релевантный товар среди рекомендованных)   

Hit rate@k = (был ли хотя бы 1 релевантный товар среди топ-k рекомендованных)

In [4]:
def hit_rate(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    hit_rate = (flags.sum() > 0) * 1
    
    return hit_rate


def hit_rate_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    k_recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, k_recommended_list)
    
    hit_rate = (flags.sum() > 0) * 1
    
    return hit_rate

In [5]:
hit_rate(recommended_list, bought_list)

1

In [6]:
hit_rate_at_k(recommended_list, bought_list, k=5)

1

In [7]:
# порядок в изначальных списках специально был изменен так, чтобы среди первых 3 рекомендованных не было купленных товаров
hit_rate_at_k(recommended_list, bought_list, k=3)

0

### 2. Precision

*Precision* - доля релевантных товаров среди рекомендованных = Какой % рекомендованных товаров  юзер купил

- Пожалуй, самая приближенная к бизнес-метрикам и самая популярная метрика

---

Precision= (# of recommended items that are relevant) / (# of recommended items)  

Precision@k = (# of recommended items @k that are relevant) / (# of recommended items @k)

Money Precision@k = (revenue of recommended items @k that are relevant) / (revenue of recommended items @k)  

**Note:** Обычно k в precision@k достаточно невелико (5-20) и определяется из бизнес-логики. Например, 5 товаров в e-mail рассылке, 20 ответов на первой странице google и т.д

Красная рыба - 400 руб  
Молоко - 60 руб  
Хлеб = 40 руб  
Гречка = 40 руб  
Шоколад = 90 руб  

------  
Варенье - 240 руб  
...  

**Case 1**  
prices_resommended = [400, 60, 40, 40 , 90]  
flags = [1, 0, 0, 0 , 1]  

$precison@5 = \frac{1 + 0 + 0 +0 + 1}{1+1+1+1+1} = 40\%$  
$money precision@5 = \frac{1*400 + 0*60 + ... + 1*90}{1*400 + 1*60 + ... + 1*90} = 77.7\%$  

  
**Case 2**   
prices_resommended = [60, 400,  40, 40 , 90]  
flags = [1, 0,  0, 0 , 1]  

$precison@5 = \frac{0 + 1 + 0 +0 + 1}{1+1+1+1+1} = 40\%$  
$money precision@5 = \frac{0*400 + 1*60 + ... + 1*90}{1*400 + 1*60 + ... + 1*90} = 15.8\%$

In [8]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision


def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision


def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
        
    # your_code
    # Лучше считать через скалярное произведение, а не цикл
    
    assert len(recommended_list) == len(prices_recommended), 'Несоответствие количества цен и товаров в рекомендации.'
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    prices_recommended = np.array(prices_recommended)
    
    k_recommended_list = recommended_list[:k]
    k_prices_recommended = prices_recommended[:k]
    
    flags = np.isin(k_recommended_list, bought_list)
    weighted_flags = flags * k_prices_recommended
    
    precision = weighted_flags.sum() / k_prices_recommended.sum()
    
    return precision

In [9]:
precision(recommended_list, bought_list)

0.2

In [10]:
precision_at_k(recommended_list, bought_list, k=5)

0.4

In [11]:
money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5)

0.6129032258064516

### 3. Recall

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

- Обычно используется для моделей пре-фильтрации товаров (убрать те товары, которые точно не будем рекомендовать)

---

Recall= (# of recommended items that are relevant) / (# of relevant items)  

Recall@k = (# of recommended items @k that are relevant) / (# of relevant items)

Money Recall@k = (revenue of recommended items @k that are relevant) / (revenue of relevant items)  

    
  
**Note:** в recall@k число k обычно достаточно большое (50-200), больше чем покупок у среднестатистического юзера

In [12]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall


def recall_at_k(recommended_list, bought_list, k=5):
    
    # your_code
    bought_list = np.array(bought_list)
    k_recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, k_recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    
    # your_code
    assert len(recommended_list) == len(prices_recommended), 'Несоответствие количества цен и товаров в рекомендации.'
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    prices_recommended = np.array(prices_recommended)
    prices_bought = np.array(prices_bought)
    
    k_recommended_list = recommended_list[:k]
    k_prices_recommended = prices_recommended[:k]
    
    flags = np.isin(k_recommended_list, bought_list)
    weighted_flags = flags * k_prices_recommended
    
    recall = weighted_flags.sum() / prices_bought.sum()
    
    return recall

In [13]:
recall(recommended_list, bought_list)

0.5

In [14]:
recall_at_k(recommended_list, bought_list, k=5)

0.5

In [15]:
money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5)

0.4523809523809524

---

# Метрики ранжирования

Если важен порядок рекомендаций. Подробнее можно почитать [здесь](https://habr.com/ru/company/econtenta/blog/303458/). Формулы в статье могут несколько отличаться от формул в лекции 

## AP@k
AP@k - average precision at k

$$AP@k = \frac{1}{r} \sum{[recommended_{relevant_i}] * precision@k}$$

- r - кол-во релевантный среди рекомендованных
- Суммируем по всем релевантным товарам
- Зависит от порядка реокмендаций

In [16]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(0, k-1):
        if flags[i] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i+1)
            sum_ += p_k
            
    result = sum_ / sum(flags)
    
    return result

In [17]:
ap_k(recommended_list, bought_list, k=5)

0.125

### MAP@k

MAP@k (Mean Average Precision@k)  
Среднее AP@k по всем юзерам
- Показывает средневзвешенную точность рекомендаций

$$MAP@k = \frac{1}{|U|} \sum_u{AP_k}$$
  
|U| - кол-во юзеров

In [18]:
recommended_list = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43],
                    [146, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43]] #id товаров
bought_list = [[521, 32, 143, 991], [146, 29, 16, 24]]

In [19]:
def map_k(recommended_list, bought_list, k=5, u=2):
    
    # your_code
    bought_list = np.array(bought_list, dtype=object)
    recommended_list = np.array(recommended_list)
    
    ap_k_sum_ = 0
    
    for j in range(u):
        flags = np.isin(recommended_list[j][:k], bought_list[j])

        if sum(flags) == 0:
            continue
            
        sum_ = 0
            
        for i in range(0, k-1):
            if flags[i]:
                p_k = precision_at_k(recommended_list[j], bought_list[j], k=i+1)
                sum_ += p_k

        ap_k_sum_ += sum_ / flags.sum()
        
    result = ap_k_sum_ / u
    
    return result

In [20]:
map_k(recommended_list, bought_list, k=5, u=2)

0.875

In [21]:
(ap_k(recommended_list[0], bought_list[0], k=5) + ap_k(recommended_list[1], bought_list[1], k=5)) / 2

0.875

### AUC@k
AUC для первых k наблюдений  
- Можно посчитать как sklern.metrics.roc_auc_score для топ-k предсказаний
- Показывает долю верно отранжированных товаров

### MRR@k
Mean Reciprocal Rank

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $k_u$
- Посчитать reciprocal rank = $\frac{1}{k_u}$

$$MRR = mean(\frac{1}{k_u})$$

[MRR](https://drive.google.com/file/d/1uMFKqWULFmehS0pJtvXKRygjYxQOPise/view?usp=sharing)

In [22]:
def reciprocal_rank(recommended_list, bought_list):
  # your_code
  result = 0
  for idx, i in enumerate(recommended_list, 1):
    if i in bought_list:
      result = 1/idx
      break
  return result

In [23]:
def mrr_at_k(recommended_list, bought_list, u, k):
  sum_ = 0
  for usr in range(u):
    rank = reciprocal_rank(recommended_list[usr][:k], bought_list[usr])
    sum_ += rank
  result = sum_ / u
  return result

In [24]:
recommended_list = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43],
                    [146, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43]] #id товаров
bought_list = [[521, 32, 143, 991], [146, 29]]

In [25]:
mrr_at_k(recommended_list, bought_list, 2, 3)

1.0

In [26]:
recommended_list = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43],
                    [146, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43]] #id товаров
bought_list = [[521, 32, 143, 991], [1134, 29]]

In [27]:
mrr_at_k(recommended_list, bought_list, 2, 3)

0.6666666666666666

### NDCG@k
Normalized discounted cumulative gain

$$DCG = \frac{1}{|r|} \sum_u{\frac{[bought fact]}{discount(i)}}$$  

$discount(i) = 1$ if $i <= 2$,   
$discount(i) = log_2(i)$ if $i > 2$


(!) Считаем для первых k рекоммендаций   
(!) - существуют вариации с другими $discount(i)$  
i - ранк рекомендованного товара  
|r| - кол-во рекомендованных товаров 

$$NDCG = \frac{DCG}{ideal DCG}$$


https://pytorch.org/docs/stable/index.html

In [28]:
import torch
torch.manual_seed(12345)
N = 5
ys_true = torch.randint(0, 5, (N, ))
ys_pred = torch.rand(N)

In [29]:
ys_true

tensor([0, 1, 0, 3, 4])

In [30]:
ys_pred

tensor([0.1784, 0.3674, 0.5676, 0.3376, 0.2119])

In [31]:
# итоговый результат модели - отсортированный ys_pred
ys_pred.sort()

torch.return_types.sort(values=tensor([0.1784, 0.2119, 0.3376, 0.3674, 0.5676]), indices=tensor([0, 4, 3, 1, 2]))

In [32]:
def compute_gain(y_value: float, gain_scheme: str) -> float:
    if gain_scheme == "exp2":
        gain = 2 ** y_value - 1
    elif gain_scheme == "const":
        gain = y_value
    else:
        raise ValueError(f"{gain_scheme} method not supported, only exp2 and const.")
    return float(gain)

In [33]:
def dcg(ys_true: torch.Tensor, ys_pred: torch.Tensor, gain_scheme: str) -> float:
    _, argsort = torch.sort(ys_pred, descending=True, dim=0)
    ys_true_sorted = ys_true[argsort]
    ret = 0
    for idx, cur_y in enumerate(ys_true_sorted, 1):
        gain = compute_gain(cur_y, gain_scheme)
        # your code
        ret += cur_y.item() / np.log2(idx+1)
    return ret

In [34]:
dcg(ys_true, ys_pred, gain_scheme='exp2')

3.85363598586503

In [35]:
def ndcg(ys_true: torch.Tensor, ys_pred: torch.Tensor, gain_scheme: str = 'const') -> float:
    pred_dcg = dcg(ys_true, ys_pred, gain_scheme)
    # your code 
    ideal_dcg = dcg(ys_true, ys_true, gain_scheme)
    ndcg = pred_dcg / ideal_dcg
    return ndcg

In [36]:
ndcg(ys_true, ys_pred, gain_scheme='exp2')

0.6028097953340635